# Transforms & DataLoaders## 🎯 Concept PrimerImage preprocessing: resize, normalize, augment (train), deterministic (val/test).**Expected:** DataLoaders with shape [B, 3, H, W]

## 📋 Objectives1. Define transforms for train/val/test2. Create custom Dataset class3. Setup DataLoaders4. Verify batch shapes

## 🔧 Setup

In [ ]:
# TODO 1: Import libraries# import torch# from torch.utils.data import Dataset, DataLoader# from torchvision import transforms# from PIL import Image

## 🔄 Define Transforms### TODO 2: Create transform pipelines**Train:** Resize, augment (flip, rotate), normalize  **Val/Test:** Resize, normalize only

In [ ]:
# TODO 2: Transforms# train_transform = transforms.Compose([#     transforms.Resize((224, 224)),#     transforms.RandomHorizontalFlip(),#     transforms.ToTensor(),#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])# ])# # val_transform = transforms.Compose([#     transforms.Resize((224, 224)),#     transforms.ToTensor(),#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])# ])

## 📦 Custom Dataset### TODO 3: Create Dataset class**Expected:** __getitem__ returns (image, label)

In [ ]:
# TODO 3: Dataset class# class RetinalDataset(Dataset):#     def __init__(self, df, img_dir, transform=None):#         self.df = df#         self.img_dir = img_dir#         self.transform = transform#     #     def __len__(self):#         return len(self.df)#     #     def __getitem__(self, idx):#         # TODO: Load image, apply transform, return label#         pass

## 🔄 DataLoaders### TODO 4: Create DataLoaders**Expected:** Batch size=32, shape [B, 3, 224, 224]

In [ ]:
# TODO 4: DataLoaders# train_dataset = RetinalDataset(train_df, img_dir, train_transform)# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)# # # Verify shape# for images, labels in train_loader:#     print(f"Batch shape: {images.shape}")#     break

## 🤔 Reflection1. Batch shape correct?2. Augmentation choices?

**Your reflection:***Write here*

## 📌 Summary✅ Transforms defined  ✅ DataLoaders ready**Next:** `03_simple_cnn_scaffold.ipynb`